# CS350 Final Project - Bias Detection and Hidden Messaging

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Get Pickles

In [ ]:
# corpus = pd.read_pickle('corpus.pkl')
# dtm = pd.read_pickle('dtm.pkl')
stm = pd.read_pickle('Stemmed.pkl')

In [ ]:
stm.head()

,label,processed_text
0,neutral,fbi crime scene investig document evid outsid ...
1,neutral,hire surg februari econom activ pick case stea...
2,neutral,manhattan district attorney conven grand juri ...
3,neutral,imag sourc afpimag caption mr merkel gave news...
4,neutral,rigger load net full balloon republican nation...


In [ ]:
corpus.head()

In [ ]:
dtm.head()

,aa,aaa,aaja,aal,aamaq,aamc,aamer,aamir,aap,aapi,...,zurawski,zurcher,zurchernorth,zurich,zusha,zushaelinsonwsjcom,zvezda,zwaan,zwick,zwirz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# corpus['label'].value_counts()

## Final Preprocessing

### LabelEncode corpus label

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
corpus['label'] = le.fit_transform(corpus['label'])

In [ ]:
# Label Encode stm
le = LabelEncoder()
stm['label'] = le.fit_transform(stm['label'])

### Check that corpus and DTM align

In [ ]:
# Show row 1 of dtm but only show columns with a value of 1 in them
# dtm.loc[1, dtm.loc[1, :] == 1]

,1
accelerates,1
accommodation,1
activity,1
adecco,1
agothe,1
...,...
weather,1
winter,1
worst,1
yeardespite,1


In [ ]:
# View the full text in row one of the corpus
# if "adecco" in corpus.loc[1, 'content']:
#   print("Corpus and DTM align!")

Corpus and DTM align!


In [ ]:
# Show all rows in dtm that contain adecco
# dtm[dtm['adecco'] == 1] # Odds of another article with 'adecco' being first by coincidence are so small

,aa,aaa,aaja,aal,aamaq,aamc,aamer,aamir,aap,aapi,...,zurawski,zurcher,zurchernorth,zurich,zusha,zushaelinsonwsjcom,zvezda,zwaan,zwick,zwirz
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4452,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4934,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8518,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11465,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11610,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Adding label to DTM

In [ ]:
dtm['__label__'] = corpus['label']

In [ ]:
dtm.tail()

,aa,aaa,aaja,aal,aamaq,aamc,aamer,aamir,aap,aapi,...,zurcher,zurchernorth,zurich,zusha,zushaelinsonwsjcom,zvezda,zwaan,zwick,zwirz,__label__
22357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
22358,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
22359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22361,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
corpus.tail()

,content,label
22357,applaud governor perrys recent decision deploy...,1
22358,voted favor hr financial services general gove...,1
22359,taken posted wokv interview congressman yoho s...,0
22360,join next town hall ocala ill answer questions...,0
22361,foreign affairs committee hearing syria remain...,0


### Downsampling (TODO: Consider upsampling as well)

In [ ]:
len(dtm[dtm['__label__'] == 0])

NameError: name 'dtm' is not defined

In [ ]:
biased_data = dtm[dtm['__label__'] == 1].sample(n=len(dtm[dtm['__label__'] == 0]), random_state=101)

In [ ]:
dtm = pd.concat([dtm[dtm['__label__'] == 0], biased_data]).sample(frac=1, random_state=101).reset_index(drop=True)

#### Downsampling and Upsampling stm

In [ ]:
stm['label'].value_counts()

,count
label,
1,14677
0,7685


In [ ]:
majority = stm[stm['label'] == 1]
minority = stm[stm['label'] == 0]

In [ ]:
middle = (len(majority) + len(minority))/2
print(middle)

11181.0


In [ ]:
from sklearn.utils import resample

majority_downsampled = resample(majority, replace=False, n_samples=int(middle), random_state=101)

minority_upsampled = resample(minority, replace=True, n_samples=int(middle), random_state=101)

In [ ]:
stm = pd.concat([majority_downsampled, minority_upsampled])

stm = stm.sample(frac=1, random_state=101).reset_index(drop=True)

In [ ]:
stm['label'].value_counts()

,count
label,
0,11181
1,11181


### Converting Stemmed Corpus to DTM

In [ ]:
stm.head()

,label,processed_text
0,neutral,fbi crime scene investig document evid outsid ...
1,neutral,hire surg februari econom activ pick case stea...
2,neutral,manhattan district attorney conven grand juri ...
3,neutral,imag sourc afpimag caption mr merkel gave news...
4,neutral,rigger load net full balloon republican nation...


In [ ]:
# Use Bag of Words on stm
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')

sdtm = cv.fit_transform(stm['processed_text'])

In [ ]:
# Pickle CountVectorizer
import pickle

with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(cv, file)

## Modeling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dtm.drop('__label__', axis=1), dtm['__label__'], test_size=0.2, random_state=101)

In [ ]:
# Stemmed dtm
X = sdtm
y = stm['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

nb.fit(X_train, y_train)

print(nb.score(X_test, y_test))
print()
print(confusion_matrix(y_test, nb.predict(X_test)))
print()
print(classification_report(y_test, nb.predict(X_test)))

0.7717415604739548

[[1685  582]
 [ 439 1767]]

              precision    recall  f1-score   support

           0       0.79      0.74      0.77      2267
           1       0.75      0.80      0.78      2206

    accuracy                           0.77      4473
   macro avg       0.77      0.77      0.77      4473
weighted avg       0.77      0.77      0.77      4473



In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

nb.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# Un-downsampled was better - TODO: try upsampling as well as downsampling
nb.score(X_test, y_test)

0.7234518220433713

In [ ]:
print(confusion_matrix(y_test, nb.predict(X_test)))
print(classification_report(y_test, nb.predict(X_test)))

[[ 865  680]
 [ 557 2371]]
              precision    recall  f1-score   support

           0       0.61      0.56      0.58      1545
           1       0.78      0.81      0.79      2928

    accuracy                           0.72      4473
   macro avg       0.69      0.68      0.69      4473
weighted avg       0.72      0.72      0.72      4473



In [ ]:
dtm['__label__'].value_counts()

,count
__label__,
1,14677
0,7685


#### Naive Bayes with Stemmed DTM

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()

nb.fit(X_train, y_train)

MultinomialNB()

In [ ]:
nb.score(X_test, y_test)

0.7717415604739548

### CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Input, Reshape
from scipy.sparse import issparse

In [ ]:
X = dtm.drop('__label__', axis=1)
y = dtm['__label__']

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# Apply feature selection
sel_k = 10000
selector = SelectKBest(chi2, k=sel_k)
X_new = selector.fit_transform(X, y)

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Apply dimensionality reduction
in_size = 1500 # DO NOT DO 5000 OR HIGHER
svd = TruncatedSVD(n_components=in_size)
X_new = svd.fit_transform(X_new)

In [ ]:
X_new.shape

(22362, 1500)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, input_dim=in_size, activation='relu'))  # Adjust input_dim to the reduced size
model.add(Dropout(0.5))  # Dropout to prevent overfitting

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.6304 - loss: 0.6738 - val_accuracy: 0.8030 - val_loss: 0.5069
Epoch 2/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7913 - loss: 0.4814 - val_accuracy: 0.8296 - val_loss: 0.4214
Epoch 3/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8335 - loss: 0.3950 - val_accuracy: 0.8343 - val_loss: 0.4010
Epoch 4/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8540 - loss: 0.3474 - val_accuracy: 0.8404 - val_loss: 0.3881
Epoch 5/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8680 - loss: 0.3201 - val_accuracy: 0.8357 - val_loss: 0.3950
Epoch 6/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8812 - loss: 0.2820 - val_accuracy: 0.8381 - val_loss: 0.3967
Epoch 7/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8989 - loss: 0.2474 - val_accuracy: 0.8428 - val_loss: 0.4175
Epoch 8/20
560/560 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9094 - loss: 0.2276 - val_accuracy: 0.

In [ ]:
# Model evaluation (sel_k = 10000, size = 1500, epochs=20)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8388 - loss: 0.5908
Test Loss: 0.5957105755805969, Test Accuracy: 0.8359043002128601


In [ ]:
# Model evaluation (sel_k = 20000, size = 1500, epochs=10)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8458 - loss: 0.4745
Test Loss: 0.4839845299720764, Test Accuracy: 0.8381399512290955


In [ ]:
# Model evaluation (sel_k = 20000, size = 1000, epochs=10)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8351 - loss: 0.4224
Test Loss: 0.4247629940509796, Test Accuracy: 0.8338922262191772


In [ ]:
# Model evaluation (sel_k = 10000, size = 2000, epochs=10)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8350 - loss: 0.5208
Test Loss: 0.5287554264068604, Test Accuracy: 0.8307623267173767


In [ ]:
# Model evaluation (sel_k = 10000, size = 1000, epochs=10)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8367 - loss: 0.4501
Test Loss: 0.45022791624069214, Test Accuracy: 0.8367985486984253


In [ ]:
# Model evaluation (sel_k = 10000, size = 300, epochs=10)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8259 - loss: 0.4276
Test Loss: 0.4406408369541168, Test Accuracy: 0.8204784393310547


#### Experimenting with ANN and sdtm

In [ ]:
# Stemmed dtm
X = sdtm
y = stm['label']

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# Apply feature selection
sel_k = 10000
selector = SelectKBest(chi2, k=sel_k)
X_new = selector.fit_transform(X, y)

In [ ]:
from sklearn.decomposition import TruncatedSVD

# Apply dimensionality reduction
in_size = 1200 # DO NOT DO 5000 OR HIGHER
svd = TruncatedSVD(n_components=in_size)
X_new = svd.fit_transform(X_new)

In [ ]:
# import pickle

# # Save the objects
# with open('preprocessing.pkl', 'wb') as file:
#     pickle.dump({'selector': selector, 'svd': svd}, file)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=101)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(128, input_dim=in_size, activation='relu'))  # Adjust input_dim to the reduced size
model.add(Dropout(0.5))  # Dropout to prevent overfitting

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
560/560 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.6202 - loss: 0.6991 - val_accuracy: 0.7863 - val_loss: 0.4835
Epoch 2/30
560/560 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7839 - loss: 0.4854 - val_accuracy: 0.8348 - val_loss: 0.3958
Epoch 3/30
560/560 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8410 - loss: 0.3762 - val_accuracy: 0.8522 - val_loss: 0.3655
Epoch 4/30
560/560 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.8722 - loss: 0.3175 - val_accuracy: 0.8672 - val_loss: 0.3373
Epoch 5/30
560/560 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8918 - loss: 0.2760 - val_accuracy: 0.8766 - val_loss: 0.3260
Epoch 6/30
560/560 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9100 - loss: 0.2384 - val_accuracy: 0.8770 - val_loss: 0.3265
Epoch 7/30
560/560 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9256 - loss: 0.1927 - val_accuracy: 0.8853 - val_loss: 0.3249
Epoch 8/30
560/560 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9355 - loss: 0.1778 - val_accuracy: 0.

In [ ]:
# Pickle the model
import pickle

with open('ANN.pkl', 'wb') as file:
    pickle.dump(model, file)

In [ ]:
# Model evaluation (sel_k = 10000, size = 1500, epochs=30)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9092 - loss: 0.3836
Test Loss: 0.40631991624832153, Test Accuracy: 0.9036440849304199


In [ ]:
# Model evaluation (sel_k = 10000, size = 1500, epochs=20)
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8993 - loss: 0.3641
Test Loss: 0.3897804915904999, Test Accuracy: 0.8938072919845581


### BERT

In [ ]:
# Use BERT
scorp = pd.read_pickle('Stemmed.pkl')

In [ ]:
le = LabelEncoder()
scorp['label'] = le.fit_transform(scorp['label'])

In [ ]:
majority = scorp[scorp['label'] == 1]
minority = scorp[scorp['label'] == 0]

In [ ]:
middle = (len(majority) + len(minority))/2
print(middle)

11181.0


In [ ]:
from sklearn.utils import resample

majority_downsampled = resample(majority, replace=False, n_samples=int(middle), random_state=101)

minority_upsampled = resample(minority, replace=True, n_samples=int(middle), random_state=101)

In [ ]:
scorp = pd.concat([majority_downsampled, minority_upsampled])

scorp = stm.sample(frac=1, random_state=101).reset_index(drop=True)

In [ ]:
scorp['label'].value_counts()

,count
label,
0,11181
1,11181


In [ ]:
scorp.head()

,label,processed_text
0,0,yesterday pleasur meet talk futur leader natio...
1,1,presid take step right securethebord safe retu...
2,1,jussi smollettin space two month empir star go...
3,1,republican lawmak taken first step strike key ...
4,0,senat republican began process tuesday shut af...


In [ ]:
# Tokenizing
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_text(texts, labels, tokenizer, max_length=128):
  encoding = tokenizer(
      texts,
      padding=True,
      truncation=True,
      max_length=max_length,
      return_tensors='pt'
  )

  return encoding['input_ids'], encoding['attention_mask'], labels


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# Splitting
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scorp['processed_text'], scorp['label'], test_size=0.2, random_state=101)

In [ ]:
# Model Setup
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
     num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Preprocess data
input_ids, attention_masks, y_train = preprocess_text(X_train.tolist(), y_train.tolist(), tokenizer) # Changed here
input_ids_val, attention_masks_val, y_test = preprocess_text(X_test.tolist(), y_test.tolist(), tokenizer) # Changed here

In [ ]:
# Create DataLoader
train_data = TensorDataset(input_ids, attention_masks, torch.tensor(y_train))
val_data = TensorDataset(input_ids_val, attention_masks_val, torch.tensor(y_test))

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)

In [ ]:
from transformers import AdamW

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Loss function
loss_fn = torch.nn.CrossEntropyLoss()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_scheduler

epochs = 10


# Scheduler for learning rate decay
num_training_steps = len(train_loader) * epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = [item.to(device) for item in batch]
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1} completed.")


Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Epoch 4 completed.
Epoch 5 completed.
Epoch 6 completed.
Epoch 7 completed.
Epoch 8 completed.
Epoch 9 completed.
Epoch 10 completed.


In [ ]:
# 10 Epochs

from sklearn.metrics import classification_report

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = [item.to(device) for item in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, axis=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, predictions))
print(confusion_matrix(true_labels, predictions))
# Accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")


              precision    recall  f1-score   support

           0       0.90      0.92      0.91      2246
           1       0.92      0.89      0.91      2227

    accuracy                           0.91      4473
   macro avg       0.91      0.91      0.91      4473
weighted avg       0.91      0.91      0.91      4473

[[2077  169]
 [ 234 1993]]
Accuracy: 0.9099038676503465


In [ ]:
# 6 Epochs

from sklearn.metrics import classification_report

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = [item.to(device) for item in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, axis=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, predictions))
print(confusion_matrix(true_labels, predictions))
# Accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")


              precision    recall  f1-score   support

           0       0.89      0.93      0.91      2246
           1       0.92      0.88      0.90      2227

    accuracy                           0.90      4473
   macro avg       0.91      0.90      0.90      4473
weighted avg       0.91      0.90      0.90      4473

[[2081  165]
 [ 260 1967]]
Accuracy: 0.90498546836575


In [ ]:
# With 3 epochs

from sklearn.metrics import classification_report

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, labels = [item.to(device) for item in batch]
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, axis=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print(classification_report(true_labels, predictions))


              precision    recall  f1-score   support

           0       0.87      0.89      0.88      2246
           1       0.89      0.87      0.88      2227

    accuracy                           0.88      4473
   macro avg       0.88      0.88      0.88      4473
weighted avg       0.88      0.88      0.88      4473



In [ ]:
print(confusion_matrix(true_labels, predictions))
# Accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy: {accuracy}")

[[2005  241]
 [ 298 1929]]
Accuracy: 0.8794992175273866
